In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import xgboost as xgb

In [2]:
data = pd.read_csv('K:/Google Drive/DOUTORADO/Tese 2.0/Chapter I/KELLOGs/dataset.csv')

Elements = ['As', 'Ba', 'Cd', 'Co', 'Cr', 'Cu', 'Pb', 'Zn', 'Mo']

for col in Elements:
    data = data.sort_values(by=col,ascending=False)
    data = data[5:]

In [3]:
def prediction(i, data):

    SEED = 255
    np.random.seed(SEED)

    Class = pd.DataFrame()
    Class['Class'] = pd.cut(data[i],
                               bins = [data[i].min(), data[i].quantile(0.75), data[i].max()],
                               labels = [0, 1],
                               right = False)

    Class = pd.concat([Class, data.iloc[:,10:]], axis=1)
    x = Class.dropna().drop(columns=['Class'])
    y = Class['Class'].dropna()

    x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                            test_size=0.2,
                                                            shuffle=True,
                                                            stratify=y)
    decision = DecisionTreeClassifier()
    decision.fit(x_train, y_train)
    y_pred_decision = decision.predict(x_test)
    
    Random = RandomForestClassifier()
    Random.fit(x_train, y_train)
    y_pred_random = Random.predict(x_test)

    xgb_classifier = xgb.XGBClassifier()
    xgb_classifier.fit(x_train, y_train)
    y_pred_xgb = xgb_classifier.predict(x_test)
    
    decision_accuracy = accuracy_score(y_test, y_pred_decision) * 100
    decision_precision = precision_score(y_test, y_pred_decision)
    decision_recall = recall_score(y_test, y_pred_decision)
    decision_f1 = f1_score(y_test, y_pred_decision)
    print(f"A acurácia para {i} usando DecisionTree foi de %.2f%%" % decision_accuracy)
    
    random_accuracy = accuracy_score(y_test, y_pred_random) * 100
    random_precision = precision_score(y_test, y_pred_random)
    random_recall = recall_score(y_test, y_pred_random)
    random_f1 = f1_score(y_test, y_pred_random)
    print(f"A acurácia para {i} usando RandomForest foi de %.2f%%" % random_accuracy)
    
    xgb_accuracy = accuracy_score(y_test, y_pred_xgb) * 100
    xgb_precision = precision_score(y_test, y_pred_xgb)
    xgb_recall = recall_score(y_test, y_pred_xgb)
    xgb_f1 = f1_score(y_test, y_pred_xgb)
    print(f"A acurácia para {i} usando XGBoost foi de %.2f%%" % xgb_accuracy)
    
    metrics_df = pd.DataFrame({
        'Element': i,
        'DecisionTree_Accuracy': [decision_accuracy],
        'DecisionTree_Precision': [decision_precision],
        'DecisionTree_Recall': [decision_recall],
        'DecisionTree_F1': [decision_f1],
        'RandomForest_Accuracy': [random_accuracy],
        'RandomForest_Precision': [random_precision],
        'RandomForest_Recall': [random_recall],
        'RandomForest_F1': [random_f1],
        'XGBoost_Accuracy': [xgb_accuracy],
        'XGBoost_Precision': [xgb_precision],
        'XGBoost_Recall': [xgb_recall],
        'XGBoost_F1': [xgb_f1]
    })

    #metrics_df.columns = [f'{i}_{col}' for col in metrics_df.columns]

    return metrics_df

In [4]:
def xgb_finetuning(i, data):
    
    SEED = 255
    np.random.seed(SEED)

    Class = pd.DataFrame()
    Class['Class'] = pd.cut(data[i],
                               bins = [data[i].min(), data[i].quantile(0.75), data[i].max()],
                               labels = [0, 1],
                               right = False)

    Class = pd.concat([Class, data.iloc[:,10:]], axis=1)
    x = Class.dropna().drop(columns=['Class'])
    y = Class['Class'].dropna()

    x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                            test_size=0.2,
                                                            shuffle=True,
                                                            stratify=y)
    
    xgb_classifier = xgb.XGBClassifier()

    param_grid = {
        'max_depth': [4, 6, 8],
        'learning_rate': [0.1, 0.2, 0.3],
        'subsample': [0.5, 1.0],
        'gamma': [0, 0.1]
    }

    grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, scoring='accuracy', cv=5)
    grid_search.fit(x_train, y_train)

    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_
    
    y_pred = best_model.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Extract TP, TN, FP, FN from the confusion matrix
    TP, FN, FP, TN = conf_matrix.ravel()
    
    # Calculate precision, recall, specificity, and F1 score
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    specificity = TN / (TN + FP)
    f1 = f1_score(y_test, y_pred)
    

    print(f"Accuracy for {i}: %.2f%%" % (accuracy * 100.0))

    print(f"Best Parameters for {i}: ", best_params)

    return best_params, best_model, accuracy, precision, recall, specificity, f1

In [5]:
result_df = pd.DataFrame(columns=['Element', 'DecisionTree_Accuracy', 'DecisionTree_Precision',
                                   'DecisionTree_Recall', 'DecisionTree_F1', 'RandomForest_Accuracy',
                                   'RandomForest_Precision', 'RandomForest_Recall', 'RandomForest_F1',
                                   'XGBoost_Accuracy', 'XGBoost_Precision', 'XGBoost_Recall',
                                   'XGBoost_F1'])

for i in Elements:
    element_df = prediction(i, data)
    result_df = pd.concat([result_df, element_df], 
                          axis=0, 
                          ignore_index=True, 
                          join='inner')

result_df.to_csv('general_pred_results.csv')

A acurácia para As usando DecisionTree foi de 81.85%
A acurácia para As usando RandomForest foi de 86.87%
A acurácia para As usando XGBoost foi de 86.49%
A acurácia para Ba usando DecisionTree foi de 86.49%
A acurácia para Ba usando RandomForest foi de 88.80%
A acurácia para Ba usando XGBoost foi de 91.89%
A acurácia para Cd usando DecisionTree foi de 83.01%
A acurácia para Cd usando RandomForest foi de 92.28%
A acurácia para Cd usando XGBoost foi de 91.51%
A acurácia para Co usando DecisionTree foi de 80.69%
A acurácia para Co usando RandomForest foi de 84.94%
A acurácia para Co usando XGBoost foi de 86.49%
A acurácia para Cr usando DecisionTree foi de 81.85%
A acurácia para Cr usando RandomForest foi de 88.42%
A acurácia para Cr usando XGBoost foi de 89.19%
A acurácia para Cu usando DecisionTree foi de 83.01%
A acurácia para Cu usando RandomForest foi de 90.73%
A acurácia para Cu usando XGBoost foi de 90.73%
A acurácia para Pb usando DecisionTree foi de 81.08%
A acurácia para Pb usan

In [8]:
data = pd.read_csv('K:/Google Drive/DOUTORADO/Tese 2.0/Chapter I/KELLOGs/dataset.csv')

Elements = ['As', 'Ba', 'Cd', 'Co', 'Cr', 'Cu', 'Pb', 'Zn', 'Mo']

for col in Elements:
    data = data.sort_values(by=col,ascending=False)
    data = data[5:]

topsoil = pd.read_csv('topsoil.csv', header=None)

data = data[data['id.layer_uuid_c'].isin(topsoil[0])]

In [9]:
result_df = pd.DataFrame(columns=['Element', 'DecisionTree_Accuracy', 'DecisionTree_Precision',
                                   'DecisionTree_Recall', 'DecisionTree_F1', 'RandomForest_Accuracy',
                                   'RandomForest_Precision', 'RandomForest_Recall', 'RandomForest_F1',
                                   'XGBoost_Accuracy', 'XGBoost_Precision', 'XGBoost_Recall',
                                   'XGBoost_F1'])

for i in Elements:
    element_df = prediction(i, data)
    result_df = pd.concat([result_df, element_df], 
                          axis=0, 
                          ignore_index=True, 
                          join='inner')

result_df.to_csv('topsoil_pred_results.csv')

A acurácia para As usando DecisionTree foi de 73.27%
A acurácia para As usando RandomForest foi de 84.16%
A acurácia para As usando XGBoost foi de 86.14%
A acurácia para Ba usando DecisionTree foi de 84.16%
A acurácia para Ba usando RandomForest foi de 91.09%
A acurácia para Ba usando XGBoost foi de 89.11%
A acurácia para Cd usando DecisionTree foi de 86.14%
A acurácia para Cd usando RandomForest foi de 94.06%
A acurácia para Cd usando XGBoost foi de 93.07%
A acurácia para Co usando DecisionTree foi de 80.20%
A acurácia para Co usando RandomForest foi de 86.14%
A acurácia para Co usando XGBoost foi de 87.13%
A acurácia para Cr usando DecisionTree foi de 81.19%
A acurácia para Cr usando RandomForest foi de 84.16%
A acurácia para Cr usando XGBoost foi de 84.16%
A acurácia para Cu usando DecisionTree foi de 91.09%
A acurácia para Cu usando RandomForest foi de 96.04%
A acurácia para Cu usando XGBoost foi de 95.05%
A acurácia para Pb usando DecisionTree foi de 82.18%
A acurácia para Pb usan

In [10]:
data = pd.read_csv('K:/Google Drive/DOUTORADO/Tese 2.0/Chapter I/KELLOGs/dataset.csv')

Elements = ['As', 'Ba', 'Cd', 'Co', 'Cr', 'Cu', 'Pb', 'Zn', 'Mo']

for col in Elements:
    data = data.sort_values(by=col,ascending=False)
    data = data[5:]

subsoil = pd.read_csv('subsoil.csv', header=None)

data = data[data['id.layer_uuid_c'].isin(subsoil[0])]

In [11]:
result_df = pd.DataFrame(columns=['Element', 'DecisionTree_Accuracy', 'DecisionTree_Precision',
                                   'DecisionTree_Recall', 'DecisionTree_F1', 'RandomForest_Accuracy',
                                   'RandomForest_Precision', 'RandomForest_Recall', 'RandomForest_F1',
                                   'XGBoost_Accuracy', 'XGBoost_Precision', 'XGBoost_Recall',
                                   'XGBoost_F1'])

for i in Elements:
    element_df = prediction(i, data)
    result_df = pd.concat([result_df, element_df], 
                          axis=0, 
                          ignore_index=True, 
                          join='inner')

result_df.to_csv('subsoil_pred_results.csv')

A acurácia para As usando DecisionTree foi de 85.96%
A acurácia para As usando RandomForest foi de 90.35%
A acurácia para As usando XGBoost foi de 92.11%
A acurácia para Ba usando DecisionTree foi de 76.32%
A acurácia para Ba usando RandomForest foi de 82.46%
A acurácia para Ba usando XGBoost foi de 79.82%
A acurácia para Cd usando DecisionTree foi de 85.96%
A acurácia para Cd usando RandomForest foi de 95.61%
A acurácia para Cd usando XGBoost foi de 94.74%
A acurácia para Co usando DecisionTree foi de 77.19%
A acurácia para Co usando RandomForest foi de 86.84%
A acurácia para Co usando XGBoost foi de 81.58%
A acurácia para Cr usando DecisionTree foi de 77.19%
A acurácia para Cr usando RandomForest foi de 83.33%
A acurácia para Cr usando XGBoost foi de 84.21%
A acurácia para Cu usando DecisionTree foi de 78.07%
A acurácia para Cu usando RandomForest foi de 86.84%
A acurácia para Cu usando XGBoost foi de 84.21%
A acurácia para Pb usando DecisionTree foi de 83.33%
A acurácia para Pb usan

In [12]:
# xgb_results_df = pd.DataFrame(columns=['Element', 'Best Parameters', 'Best Model', 'Accuracy', 'Precision','Recall','Specificity','F1 Score'])

# for i in Elements:
#     best_params, best_model, accuracy, precision, recall, specificity, f1 = xgb_finetuning(i, data)
#     dictionary = {'Element': i, 'Best Parameters': best_params, 'Best Model': best_model, 'Accuracy': accuracy,
#                   'Precision': precision, 'Recall': recall,'Specificity': specificity, 'F1 Score': f1}
#     dataframe = pd.DataFrame(dictionary)
    
#     xgb_results_df = pd.concat([xgb_results_df, dataframe], 
#                           axis=0, 
#                           ignore_index=True, 
#                           join='inner')
    
# xgb_results_df.to_csv('xgb_results.csv')